# Data Processing
## Weighted Average Download Speed at the Country level
Using Ookla's opensource dataset we calculate an average download speed weighted by population for each country quarterly from 2019 till 2nd quarter of 2025.

## Methodology
1. Filter Ookla's data by country. 
2. Aggregate Ookla's data to the desired zoom level. The zoom level should be smaller than 16 which is the level at which Ookla delivers their data. This step is optional if you want to work with 16 agregation level
3. Calculate an average speed for each tile by averaging the speeds at lower level tiles. Tiles with no speed as disregarded from the average
4. Aggregate Worldpop data to each tile, this results in an estimate of population at each tile.
5. Calculate an average download weighted by population as follows
$$
WeightedAvgSpeed_{tile_{i}} = \frac{Pop_{i} * AvgSpeed_{tile_{i}}}{\sum_{i=1}^{m}Pop_{i}} 
$$


In [1]:
import pandas as pd
import mercantile
from shapely.geometry import box
import geopandas as gpd
from concurrent.futures import ThreadPoolExecutor, as_completed
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
import rasterio
from rasterstats import zonal_stats
import glob
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import product
import os
from shapely import Point

In [2]:
path_data = '/home/sol/gitrepo/MENA-FCV-economic-monitor/data/'
zoom = 12
net_type = 'fixed'

In [3]:
iso_codes = [
    "afg", "are", "bhr", "dji", "dza", "egy", "irn", "irq", "jor", "kwt",
    "lbn", "lby", "mar", "omn", "pak", "pse", "qat", "sau", "syr", "tun", "yem"]
iso_codes = [x.upper() for x in iso_codes]

## Obtain Quadkeys for each country

In [4]:
def tile_to_quadkey(x, y, z):
    quadkey = ''
    for i in range(z, 0, -1):
        digit = 0
        mask = 1 << (i - 1)
        if (x & mask) != 0:
            digit += 1
        if (y & mask) != 0:
            digit += 2
        quadkey += str(digit)
    return quadkey

In [5]:
def process_tile(tile):
    qk = tile_to_quadkey(tile.x, tile.y, tile.z)
    geom = box(*mercantile.bounds(tile))
    return qk, geom

In [6]:
def get_quadkeys_bbox_country(geo_data, zoom):
    geom = geo_data.geometry
    bounds = geom.bounds
    tiles = list(mercantile.tiles(bounds[0], bounds[1], bounds[2], bounds[3], zoom))
    results = []
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = [executor.submit(process_tile, tile) for tile in tiles]
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing tiles"):
            results.append(future.result())
    df = pd.DataFrame({qk: {"geometry": geom} for qk, geom in results}).T
    gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")
    return gdf
    
def get_quadkeys_country(boundary, zoom):
    gdf = get_quadkeys_bbox_country(boundary.loc[0], zoom)
    country_quadkeys = gpd.sjoin(gdf, boundary, how='inner', predicate='intersects')
    return country_quadkeys[['geometry']]

### Obtain GDF at level 12 per country

In [ ]:
countries_gdf = {}
for iso_code in iso_codes: 
    boundary = gpd.read_file(path_data + f'admin_boundaries/{iso_code}_ADM0_gbOpen.geojson')
    gdf = get_quadkeys_country(boundary, zoom)
    countries_gdf[iso_code] = gdf
    gdf.to_file(f'../results/gdf_{iso_code}.gpkg')

Processing tiles: 100%|███████████████| 20955/20955 [00:00<00:00, 158992.70it/s]


### Obtain Quadkeys level 16 per country to filter Ookla's data

In [4]:
zoom_internet = 16

In [5]:
def get_subquadkeys(quadkey, target_level):
    """Return all quadkeys at target_level within the given quadkey"""
    delta = target_level - len(quadkey)
    if delta < 0:
        raise ValueError("Target level must be greater than or equal to the current quadkey level")
    sub_quadkeys = []
    for suffix in product('0123', repeat=delta):
        sub_quadkeys.append(quadkey + ''.join(suffix))
    return sub_quadkeys

In [6]:
def get_all_childquadkeys(parent_quadkeys, target_level):
    child_quadkeys = []
    for parent_qk in parent_quadkeys:
        sub_qks = get_subquadkeys(parent_qk, target_level)
        child_quadkeys+=sub_qks
    return child_quadkeys

In [7]:
def quadkey_to_centroid(quadkey):
    tile = mercantile.quadkey_to_tile(quadkey)
    bounds = mercantile.bounds(tile)
    lon = (bounds.west + bounds.east) / 2
    lat = (bounds.south + bounds.north) / 2
    return Point(lon, lat)

In [8]:
def filter_quadkeys_country(iso_code):
    tiles = gpd.read_file(f'../results/gdf_{iso_code}.gpkg')
    child_quadkeys = get_all_childquadkeys(tiles['index'].tolist(), 16)
    gdf = gpd.GeoDataFrame({
        'quadkey': child_quadkeys,
        'geometry': [quadkey_to_centroid(qk) for qk in child_quadkeys]
        }, crs="EPSG:4326") 
    boundary = gpd.read_file(path_data + f'admin_boundaries/{iso_code}_ADM0_gbOpen.geojson')
    country_quadkeys = gpd.sjoin(gdf, boundary, how='inner', predicate='within')
    df_qks = pd.DataFrame(index = [], columns = [])
    df_qks['quadkey'] = country_quadkeys.quadkey.tolist()
    df_qks['country'] = iso_code
    return df_qks

In [9]:
with ProcessPoolExecutor() as executor:
    results = executor.map(filter_quadkeys_country, iso_codes)    

ookla_qks = []
for df_qks in results:
    ookla_qks.append(df_qks)
pd.concat(ookla_qks).to_csv('../results/quadkeys_per_country.csv')

## Obtain Ookla + Worldpop data by country

### Aggregate WorldPop at Grid level

In [8]:
def process_country(args):
    country, gdf, path_data = args
    try:
        path_raster = path_data + f'worldpop/{country.lower()}_ppp_2020_UNadj_constrained.tif'
        print(path_raster)
        raster_stats = zonal_stats(gdf, path_raster, stats=['sum'])
        pop_by_quadkey = [elem['sum'] if elem['sum'] is not None else 0 for elem in raster_stats]
        gdf['population'] = pop_by_quadkey
        return country, gdf
    except Exception as e:
        print(f'error processing {country}: {e}')
        traceback.print_exc()
        return country, None
    
inputs = [(country, gdf, path_data) for country, gdf in countries_gdf.items()]
with ProcessPoolExecutor() as executor:
    results = executor.map(process_country, inputs)

for country, gdf in results:
    countries_gdf[country] = gdf
    gdf.to_file(f'../results/gdf_{country}.gpkg')

/home/sol/gitrepo/MENA-FCV-economic-monitor/data/worldpop/bhr_ppp_2020_UNadj_constrained.tif
/home/sol/gitrepo/MENA-FCV-economic-monitor/data/worldpop/dji_ppp_2020_UNadj_constrained.tif/home/sol/gitrepo/MENA-FCV-economic-monitor/data/worldpop/are_ppp_2020_UNadj_constrained.tif

/home/sol/gitrepo/MENA-FCV-economic-monitor/data/worldpop/afg_ppp_2020_UNadj_constrained.tif
/home/sol/gitrepo/MENA-FCV-economic-monitor/data/worldpop/irq_ppp_2020_UNadj_constrained.tif
/home/sol/gitrepo/MENA-FCV-economic-monitor/data/worldpop/jor_ppp_2020_UNadj_constrained.tif
/home/sol/gitrepo/MENA-FCV-economic-monitor/data/worldpop/dza_ppp_2020_UNadj_constrained.tif
/home/sol/gitrepo/MENA-FCV-economic-monitor/data/worldpop/egy_ppp_2020_UNadj_constrained.tif
/home/sol/gitrepo/MENA-FCV-economic-monitor/data/worldpop/irn_ppp_2020_UNadj_constrained.tif
/home/sol/gitrepo/MENA-FCV-economic-monitor/data/worldpop/kwt_ppp_2020_UNadj_constrained.tif
/home/sol/gitrepo/MENA-FCV-economic-monitor/data/worldpop/lbn_ppp_2020

### Aggregate Internet results at the Grid level

### Data Processing

In [4]:
countries_gdf = {}
for iso_code in iso_codes:
    gdf=gpd.read_file(f'../results/gdf_{iso_code}.gpkg')
    gdf.set_index('index', inplace = True)
    countries_gdf[iso_code] = gdf

In [5]:
net_type = 'fixed'

In [6]:
countries_quadkeys = pd.read_csv('../results/quadkeys_per_country.csv', dtype = {'quadkey': 'str'})
countries_qk_dict = {}
for iso_code in iso_codes:
    countries_qk_dict[iso_code] = countries_quadkeys[countries_quadkeys['country']==iso_code].quadkey.tolist()

In [4]:
for year in range(2019, 2026):
    for quarter in range(1, 5):
        if (quarter > 2) & (year == 2025):
            continue
        path = path_data + f'type={net_type}/year={year}/quarter={quarter}/'
        df = pd.read_parquet(path, engine="pyarrow")
        df[f'quadkey_z{zoom}'] = df['quadkey'].apply(lambda x: x[:zoom])
        for country, gdf in countries_gdf.items():
            country_quadkeys = countries_qk_dict[country]
            df_country = df[df['quadkey'].isin(country_quadkeys)].copy()
            avg_download_speed = df_country.groupby(f'quadkey_z{zoom}')['avg_d_kbps'].mean()
            gdf[f'avg_download_{year}_{quarter}'] = avg_download_speed
            countries_gdf[country] = gdf
            gdf.to_file(f'../results/gdf_{country}_with_variables.gpkg')
